In [1]:
# !pip install langchain_chroma
# !pip install langchain_huggingface
# !pip install langchain_openai

In [2]:
# !pip install protobuf==3.20.*

In [3]:
from agent import Chatbot, Text2SQL

In [4]:
from agent.const import (
    ChatConfig,
    Text2SQLConfig,
    GEMINI_FAST_CONFIG,
    INBETWEEN_CHAT_CONFIG,
    GPT4O_MINI_CONFIG,
    TEXT2SQL_MEDIUM_OPENAI_CONFIG,
    TEXT2SQL_MEDIUM_GEMINI_CONFIG,
    TEXT2SQL_FASTEST_CONFIG,
    TEXT2SQL_SWEET_SPOT_CONFIG
)

from agent.prompt.prompt_controller import (
    PromptConfig, 
    VERTICAL_PROMPT_BASE, 
    VERTICAL_PROMPT_UNIVERSAL,
    HORIZONTAL_PROMPT_BASE,
    HORIZONTAL_PROMPT_UNIVERSAL
)

In [5]:
from ETL.dbmanager.setup import (
    DBConfig,
    BGE_VERTICAL_BASE_CONFIG,
    BGE_VERTICAL_UNIVERSAL_CONFIG,
    BGE_HORIZONTAL_BASE_CONFIG,
    OPENAI_VERTICAL_UNIVERSAL_CONFIG,
    setup_db
)

from langchain_huggingface import HuggingFaceEmbeddings
import json
import torch

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [6]:
db_config = DBConfig(**OPENAI_VERTICAL_UNIVERSAL_CONFIG)
chat_config = ChatConfig(**GEMINI_FAST_CONFIG)
text2sql_config = Text2SQLConfig(**TEXT2SQL_MEDIUM_OPENAI_CONFIG)
prompt_config = PromptConfig(**VERTICAL_PROMPT_UNIVERSAL)

In [7]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# embedding_model = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5', model_kwargs = {'device': device})
# db_config.embedding = embedding_model

db = setup_db(db_config)
text2sql = Text2SQL(config = text2sql_config, prompt_config=prompt_config, db = db, max_steps=2)

2024-12-18 22:48:10,727 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


test_db


2024-12-18 22:48:10,970 - INFO - Finish setup for Vertical Universal


In [8]:
chatbot = Chatbot(config = chat_config, text2sql = text2sql)

In [9]:
chatbot.display_history

[]

In [14]:
for c in chatbot.stream("Hi how are you"):
    print(c)

2024-12-18 22:48:54,456 - INFO - Completion time of gemini-1.5-flash-002: 1.8591060638427734s
2024-12-18 22:48:54,457 - INFO - Routing not triggered
2024-12-18 22:48:54,458 - INFO - Reasoning time with streaming: 1.8596270084381104s


prompt_token_count: 168
candidates_token_count: 15
total_token_count: 183

You
 are only allowed to ask questions related to finance and accounting.



In [11]:
chatbot.chat("Hi how are you")

2024-12-18 22:48:12,597 - INFO - Completion time of gemini-1.5-flash-002: 1.453141450881958s
2024-12-18 22:48:12,598 - INFO - Routing not triggered


prompt_token_count: 148
candidates_token_count: 15
total_token_count: 163



2024-12-18 22:48:13,937 - INFO - Completion time of gemini-1.5-flash-002: 1.3390069007873535s
2024-12-18 22:48:13,937 - INFO - Reasoning time without streaming: 2.7931480407714844s


prompt_token_count: 73
candidates_token_count: 14
total_token_count: 87



'You are only allowed to ask questions related to finance and accounting.\n'

In [12]:
chatbot.history

[{'role': 'system',
  'content': '\nYou are a financial analyst and you have access to a database of financial statements of companies from 2019 to 2024.\nOnly answer questions related to finance and accounting.\n\nIf the question is not related to finance and accounting, say You are only allowed to ask questions related to finance and accounting.\n'},
 {'role': 'user', 'content': 'Hi how are you'},
 {'role': 'assistant',
  'content': 'You are only allowed to ask questions related to finance and accounting.\n'}]

In [13]:
chatbot.display_history

[{'role': 'user', 'content': 'Hi how are you'},
 {'role': 'assistant',
  'content': 'You are only allowed to ask questions related to finance and accounting.\n'}]